In [1]:
import sys
sys.path.append('../')
sys.path.append('../hifigan')
from cube.api import TTSCube

/Users/boros/Projects/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cube=TTSCube.load('blizzard2023-hf')

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']                      
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
import IPython
audio=cube('Bonjour!§§ Je suis un system artificialle.§', speaker='neb')
IPython.display.Audio(data=audio, rate=24000)

In [4]:
audio=cube('Paris est la capitale de la France.§§ Divisée en vingt arrondissements, elle est le chef-lieu de la région Île-de-France et le siège de la métropole du Grand Paris.§§ Elle est établie au centre du Bassin parisien, sur une boucle de la Seine, entre les confluents avec la Marne et l\'Oise.', speaker='neb')
IPython.display.Audio(data=audio, rate=24000)

In [6]:
audio=cube("ALICE, assise auprès de sa sœur sur le gazon, commençait à s'ennuyer de rester là à ne rien faire; une ou deux fois elle avait jeté les yeux sur le livre que lisait sa sœur; mais quoi! pas d'images, pas de dialogues! \"La belle avance,\" pensait Alice, \"qu'un livre sans images, sans causeries!\".", speaker='neb')
IPython.display.Audio(data=audio, rate=24000)

In [ ]:
audio=cube("""Il n'y avait rien là de bien étonnant, et Alice ne trouva même pas
très-extraordinaire d'entendre parler le Lapin qui se disait: "Ah!§§
j'arriverai trop tard!"§§ (En y songeant après, il lui sembla bien
qu'elle aurait dû s'en étonner, mais sur le moment cela lui avait paru
tout naturel.) §§ Cependant, quand le Lapin vint à tirer une montre de
son gousset, la regarda, puis se prit à courir de plus belle, Alice
sauta sur ses pieds, frappée de cette idée que jamais elle n'avait vu
de lapin avec un gousset et une montre. §§ Entraînée par la curiosité elle
s'élança sur ses traces à travers le champ, et arriva tout juste à
temps pour le voir disparaître dans un large trou au pied d'une haie.""", speaker='neb')
IPython.display.Audio(data=audio, rate=24000)

In [8]:
audio=cube("«Levez-vous», dit le professeur.\n\nIl se leva; sa casquette tomba. Toute la classe se mit à rire. \n\nIl se baissa pour la reprendre. Un voisin la fit tomber d'un coup de coude, il la ramassa encore une fois. \n\n «Débarrassez-vous donc de votre casque», dit le professeur, qui était un homme d'esprit.", speaker="neb")
IPython.display.Audio(data=audio, rate=24000)   